# 1. Download Data

This notebook downloads the necessary example data that will be used in other notebooks. In particular, the notebook does the following:

- Download beer and urine .mzML files used as examples in the paper
- Download the HMDB database and extract metabolites.
- Trains kernel density estimators on the mzML files.
- Extract regions of interests from the mzML files.

**Please run this notebook first to make sure the data files are available for subsequent notebooks.**

In [1]:
%matplotlib inline

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from pathlib import Path
import glob

In [4]:
import sys
sys.path.append('..')

In [5]:
from vimms.DataGenerator import extract_hmdb_metabolite, get_data_source, get_spectral_feature_database
from vimms.MassSpec import IndependentMassSpectrometer
from vimms.Controller import SimpleMs1Controller
from vimms.Common import *
from vimms.Roi import make_roi, RoiToChemicalCreator, extract_roi

In [6]:
# set_log_level_info()
set_log_level_debug()

## a. Download beer and urine files

Here we download the beer and urine .mzML files used as examples in the paper if they don't exist.

In [7]:
url = 'http://researchdata.gla.ac.uk/870/2/example_data.zip'
base_dir = os.path.join(os.getcwd(), 'example_data')

In [8]:
if not os.path.isdir(base_dir): # if not exist then download the example data and extract it
    print('Creating %s' % base_dir)    
    out_file = 'example_data.zip'
    download_file(url, out_file)
    extract_zip_file(out_file, delete=True)
else:
    print('Found %s' % base_dir)

Found /home/cjurich/projects/vimms/examples/example_data


## b. Download metabolites from HMDB

Next we load a pre-processed pickled file of database metabolites in the `data_dir` folder. If it is not found, then create the file by downloading and extracting the metabolites from HMDB.

In [9]:
compound_file = Path(base_dir, 'hmdb_compounds.p')
hmdb_compounds = load_obj(compound_file)
if hmdb_compounds is None: # if file does not exist

    # download the entire HMDB metabolite database
    url = 'http://www.hmdb.ca/system/downloads/current/hmdb_metabolites.zip'

    out_file = download_file(url)
    compounds = extract_hmdb_metabolite(out_file, delete=True)
    save_obj(compounds, compound_file)

else:
    print('Loaded %d DatabaseCompounds from %s' % (len(hmdb_compounds), compound_file))

Loaded 114202 DatabaseCompounds from /home/cjurich/projects/vimms/examples/example_data/hmdb_compounds.p


## c. Generate Spectral Feature Database

In this section we demonstrate how ViMMS constructs the spectral feature database containing information, such as the densities of m/z, RT and intensities, scan durations, MS2 peaks, from the example Beer mzML files. The spectral feature database will be used to sample for various features during the simulation later.

The following two methods `get_data_source` and `get_spectral_feature_database` from ViMMS will be used. 
- `get_data_source` loads a `DataSource` object that stores information on a set of .mzML files
- `get_spectral_feature_database` extracts relevant features from .mzML files that have been loaded into the DataSource. 

The parameter below should work for most cases, however for different data, it might be necessary to adjust the `min_rt` and `max_rt` values.

In [10]:
filename = None                    # if None, use all mzML files found
min_ms1_intensity = 0              # min MS1 intensity threshold to include a data point for density estimation
min_ms2_intensity = 0              # min MS2 intensity threshold to include a data point for density estimation
min_rt = 0                         # min RT to include a data point for density estimation
max_rt = 1440                      # max RT to include a data point for density estimation
bandwidth_mz_intensity_rt = 1.0    # kernel bandwidth parameter to sample (mz, RT, intensity) values during simulation
bandwidth_n_peaks = 1.0            # kernel bandwidth parameter to sample number of peaks per scan during simulation

### Load fullscan data and train spectral feature database

In [11]:
mzml_path = Path(base_dir, 'beers', 'fullscan', 'mzML')
xcms_output = Path(mzml_path, 'extracted_peaks_ms1.csv')
out_file = Path(base_dir, 'peak_sampler_mz_rt_int_19_beers_fullscan.p')

In [12]:
ds_fullscan = get_data_source(mzml_path, filename, xcms_output)

INFO   : DataSource                     : Loading Beer_multibeers_7_fullscan1.mzML
INFO   : numexpr.utils                  : NumExpr defaulting to 8 threads.
INFO   : DataSource                     : Loading Beer_multibeers_6_fullscan1.mzML
INFO   : DataSource                     : Loading Beer_multibeers_19_fullscan1.mzML
INFO   : DataSource                     : Loading Beer_multibeers_4_fullscan1.mzML
INFO   : DataSource                     : Loading Beer_multibeers_10_fullscan1.mzML
INFO   : DataSource                     : Loading Beer_multibeers_17_fullscan1.mzML
INFO   : DataSource                     : Loading Beer_multibeers_3_fullscan1.mzML
INFO   : DataSource                     : Loading Beer_multibeers_8_fullscan1.mzML
INFO   : DataSource                     : Loading Beer_multibeers_16_fullscan1.mzML
INFO   : DataSource                     : Loading Beer_multibeers_12_fullscan1.mzML
INFO   : DataSource                     : Loading Beer_multibeers_13_fullscan1.mzML
INFO  

In [13]:
ps = get_spectral_feature_database(ds_fullscan, filename, min_ms1_intensity, min_ms2_intensity, min_rt, max_rt,
               bandwidth_mz_intensity_rt, bandwidth_n_peaks, out_file)

DEBUG  : PeakSampler                    : Extracted 0 MS2 scans
DEBUG  : PeakSampler                    : Computing parent intensity proportions
DEBUG  : PeakSampler                    : Extracting scan durations
DEBUG  : PeakSampler                    : Training KDEs for ms_level=1
DEBUG  : PeakSampler                    : Retrieving mz_intensity_rt values from <vimms.DataGenerator.DataSource object at 0x7f2e3ec14640>
INFO   : DataSource                     : Using values from XCMS peaklist
DEBUG  : PeakSampler                    : Retrieving n_peaks values from <vimms.DataGenerator.DataSource object at 0x7f2e3ec14640>
DEBUG  : PeakSampler                    : Training KDEs for ms_level=2
DEBUG  : PeakSampler                    : Retrieving mz_intensity_rt values from <vimms.DataGenerator.DataSource object at 0x7f2e3ec14640>
INFO   : DataSource                     : Using values from scans


Saving <class 'vimms.DataGenerator.PeakSampler'> to /home/cjurich/projects/vimms/examples/example_data/peak_sampler_mz_rt_int_19_beers_fullscan.p


In [14]:
ps.get_peak(1, 10) # try to sample 10 MS1 peaks

[Peak mz=428.7252 rt=1081.71 intensity=2123572.19 ms_level=1,
 Peak mz=411.0711 rt=602.91 intensity=12226691.90 ms_level=1,
 Peak mz=126.0966 rt=267.76 intensity=93690.81 ms_level=1,
 Peak mz=206.4157 rt=480.98 intensity=62336.55 ms_level=1,
 Peak mz=495.0441 rt=546.91 intensity=907325.70 ms_level=1,
 Peak mz=210.3872 rt=289.68 intensity=582443.59 ms_level=1,
 Peak mz=249.6236 rt=399.33 intensity=3002228.13 ms_level=1,
 Peak mz=122.2433 rt=76.53 intensity=16820.22 ms_level=1,
 Peak mz=150.6783 rt=225.97 intensity=332209.90 ms_level=1,
 Peak mz=150.3422 rt=521.56 intensity=1986196.81 ms_level=1]

### Load fragmentation data and train spectral feature database

In [15]:
mzml_path = Path(base_dir, 'beers', 'fragmentation', 'mzML')
xcms_output = Path(mzml_path, 'extracted_peaks_ms1.csv')
out_file = Path(base_dir, 'peak_sampler_mz_rt_int_19_beers_fragmentation.p')

In [16]:
ds_fragmentation = get_data_source(mzml_path, filename, xcms_output)

INFO   : DataSource                     : Loading Beer_multibeers_8_T10_POS.mzML
INFO   : DataSource                     : Loading Beer_multibeers_12_T10_POS.mzML
INFO   : DataSource                     : Loading Beer_multibeers_9_T10_POS.mzML
INFO   : DataSource                     : Loading Beer_multibeers_1_T10_POS.mzML
INFO   : DataSource                     : Loading Beer_multibeers_10_T10_POS.mzML
INFO   : DataSource                     : Loading Beer_multibeers_19_T10_POS.mzML
INFO   : DataSource                     : Loading Beer_multibeers_6_T10_POS.mzML
INFO   : DataSource                     : Loading Beer_multibeers_3_T10_POS.mzML
INFO   : DataSource                     : Loading Beer_multibeers_15_T10_POS.mzML
INFO   : DataSource                     : Loading Beer_multibeers_2_T10_POS.mzML
INFO   : DataSource                     : Loading Beer_multibeers_17_T10_POS.mzML
INFO   : DataSource                     : Loading Beer_multibeers_4_T10_POS.mzML
INFO   : DataSource    

In [17]:
ps = get_spectral_feature_database(ds_fragmentation, filename, min_ms1_intensity, min_ms2_intensity, min_rt, max_rt,
               bandwidth_mz_intensity_rt, bandwidth_n_peaks, out_file)

DEBUG  : PeakSampler                    : Extracted 138969 MS2 scans
DEBUG  : PeakSampler                    : Computing parent intensity proportions
DEBUG  : PeakSampler                    : Extracting scan durations
DEBUG  : PeakSampler                    : Training KDEs for ms_level=1
DEBUG  : PeakSampler                    : Retrieving mz_intensity_rt values from <vimms.DataGenerator.DataSource object at 0x7f2e1bef3d60>
INFO   : DataSource                     : Using values from XCMS peaklist
DEBUG  : PeakSampler                    : Retrieving n_peaks values from <vimms.DataGenerator.DataSource object at 0x7f2e1bef3d60>
DEBUG  : PeakSampler                    : Training KDEs for ms_level=2
DEBUG  : PeakSampler                    : Retrieving mz_intensity_rt values from <vimms.DataGenerator.DataSource object at 0x7f2e1bef3d60>
INFO   : DataSource                     : Using values from scans
DEBUG  : PeakSampler                    : Retrieving n_peaks values from <vimms.DataGenerat

Saving <class 'vimms.DataGenerator.PeakSampler'> to /home/cjurich/projects/vimms/examples/example_data/peak_sampler_mz_rt_int_19_beers_fragmentation.p


In [18]:
ps.get_peak(1, 10)

[Peak mz=221.7807 rt=214.06 intensity=320377.74 ms_level=1,
 Peak mz=242.0755 rt=342.85 intensity=457720.15 ms_level=1,
 Peak mz=601.3200 rt=420.90 intensity=3570.79 ms_level=1,
 Peak mz=436.5792 rt=801.95 intensity=87740.36 ms_level=1,
 Peak mz=301.6364 rt=249.27 intensity=61586.19 ms_level=1,
 Peak mz=246.0578 rt=262.69 intensity=278773.64 ms_level=1,
 Peak mz=473.5991 rt=633.66 intensity=342773.23 ms_level=1,
 Peak mz=178.9170 rt=1261.88 intensity=25533.11 ms_level=1,
 Peak mz=254.8886 rt=375.12 intensity=718706.28 ms_level=1,
 Peak mz=679.5092 rt=218.24 intensity=361896.01 ms_level=1]

In [19]:
ps.get_peak(2, 10)

[Peak mz=70.5123 rt=769.84 intensity=12155.36 ms_level=2,
 Peak mz=306.8993 rt=948.28 intensity=3623.41 ms_level=2,
 Peak mz=97.8524 rt=1016.22 intensity=3337.43 ms_level=2,
 Peak mz=103.5200 rt=342.68 intensity=1281.02 ms_level=2,
 Peak mz=111.4647 rt=543.28 intensity=21201.48 ms_level=2,
 Peak mz=118.6177 rt=437.24 intensity=86636.24 ms_level=2,
 Peak mz=85.0430 rt=1161.27 intensity=557.34 ms_level=2,
 Peak mz=272.8699 rt=253.92 intensity=36855.75 ms_level=2,
 Peak mz=94.4220 rt=368.48 intensity=1134.53 ms_level=2,
 Peak mz=52.8812 rt=923.67 intensity=1059.23 ms_level=2]

## d. Extract the ROIs for DsDA Experiments

In [20]:
roi_mz_tol = 10
roi_min_length = 2
roi_min_intensity = 1.75E5
roi_start_rt = min_rt
roi_stop_rt = max_rt

#### Extract beer ROIs

In [21]:
file_names = Path(base_dir, 'beers', 'fragmentation', 'mzML').glob('*.mzML')
out_dir = Path(base_dir,'DsDA', 'DsDA_Beer', 'beer_t10_simulator_files')
mzml_path = Path(base_dir, 'beers', 'fragmentation', 'mzML')

extract_roi(list(file_names), out_dir, 'beer_%d.p', mzml_path, ps)

DEBUG  : RoiToChemicalCreator           :      0/ 13179
INFO   : RoiToChemicalCreator           : Found 13179 ROIs above thresholds


Saving <class 'list'> to /home/cjurich/projects/vimms/examples/example_data/DsDA/DsDA_Beer/beer_t10_simulator_files/beer_8.p


DEBUG  : RoiToChemicalCreator           :      0/ 11249
INFO   : RoiToChemicalCreator           : Found 11249 ROIs above thresholds


Saving <class 'list'> to /home/cjurich/projects/vimms/examples/example_data/DsDA/DsDA_Beer/beer_t10_simulator_files/beer_12.p


DEBUG  : RoiToChemicalCreator           :      0/ 14842
INFO   : RoiToChemicalCreator           : Found 14842 ROIs above thresholds


...

Saving <class 'list'> to /home/cjurich/projects/vimms/examples/example_data/DsDA/DsDA_Beer/beer_t10_simulator_files/beer_7.p


DEBUG  : RoiToChemicalCreator           :      0/ 12840
INFO   : RoiToChemicalCreator           : Found 12840 ROIs above thresholds


Saving <class 'list'> to /home/cjurich/projects/vimms/examples/example_data/DsDA/DsDA_Beer/beer_t10_simulator_files/beer_14.p


DEBUG  : RoiToChemicalCreator           :      0/ 10502
INFO   : RoiToChemicalCreator           : Found 10502 ROIs above thresholds


Saving <class 'list'> to /home/cjurich/projects/vimms/examples/example_data/DsDA/DsDA_Beer/beer_t10_simulator_files/beer_5.p


#### Extract urine ROIs

In [22]:
file_names = Path(base_dir, 'urines', 'fragmentation', 'mzML').glob('*.mzML')
out_dir = Path(base_dir,'DsDA', 'DsDA_Urine', 'urine_t10_simulator_files')
mzml_path = Path(base_dir, 'urines', 'fragmentation', 'mzML')

extract_roi(list(file_names), out_dir, 'urine_%d.p', mzml_path, ps)

DEBUG  : RoiToChemicalCreator           :      0/ 16320
INFO   : RoiToChemicalCreator           : Found 16320 ROIs above thresholds


Created /home/cjurich/projects/vimms/examples/example_data/DsDA/DsDA_Urine/urine_t10_simulator_files
Saving <class 'list'> to /home/cjurich/projects/vimms/examples/example_data/DsDA/DsDA_Urine/urine_t10_simulator_files/urine_97.p


DEBUG  : RoiToChemicalCreator           :      0/ 16294
INFO   : RoiToChemicalCreator           : Found 16294 ROIs above thresholds


Saving <class 'list'> to /home/cjurich/projects/vimms/examples/example_data/DsDA/DsDA_Urine/urine_t10_simulator_files/urine_85.p


DEBUG  : RoiToChemicalCreator           :      0/ 16321
INFO   : RoiToChemicalCreator           : Found 16321 ROIs above thresholds


...

Saving <class 'list'> to /home/cjurich/projects/vimms/examples/example_data/DsDA/DsDA_Urine/urine_t10_simulator_files/urine_17.p


DEBUG  : RoiToChemicalCreator           :      0/ 17858
INFO   : RoiToChemicalCreator           : Found 17858 ROIs above thresholds


Saving <class 'list'> to /home/cjurich/projects/vimms/examples/example_data/DsDA/DsDA_Urine/urine_t10_simulator_files/urine_52.p


DEBUG  : RoiToChemicalCreator           :      0/ 13999
INFO   : RoiToChemicalCreator           : Found 13999 ROIs above thresholds


Saving <class 'list'> to /home/cjurich/projects/vimms/examples/example_data/DsDA/DsDA_Urine/urine_t10_simulator_files/urine_18.p
